In [1]:
import os
from dotenv import load_dotenv

load_dotenv()


GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [3]:
loader_pdf = PyPDFDirectoryLoader(
    "../SafetyAgentKnowledgeBases"
)

docs_pdf = loader_pdf.load()
len(docs_pdf)

186

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs_pdf)

len(texts)

466

In [8]:
pip install -U langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True}

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

In [10]:
from langchain_chroma import Chroma

persist_directory = 'safety_kb_db'

embedding = bge_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [11]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [12]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [13]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="qwen/qwen3-32b",
        )